<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Modeling" data-toc-modified-id="Modeling-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#3.-데이터-구성하기" data-toc-modified-id="3.-데이터-구성하기-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>3. 데이터 구성하기</a></span><ul class="toc-item"><li><span><a href="#(1)-명목형-데이터의-인덱싱-표-만들기" data-toc-modified-id="(1)-명목형-데이터의-인덱싱-표-만들기-1.0.1.1"><span class="toc-item-num">1.0.1.1&nbsp;&nbsp;</span>(1) 명목형 데이터의 인덱싱 표 만들기</a></span></li><li><span><a href="#(2)-각-column의-값을-embedding-matrix의-index로-변경하기" data-toc-modified-id="(2)-각-column의-값을-embedding-matrix의-index로-변경하기-1.0.1.2"><span class="toc-item-num">1.0.1.2&nbsp;&nbsp;</span>(2) 각 column의 값을 embedding matrix의 index로 변경하기</a></span></li><li><span><a href="#(3)-train-set과-test-set으로-구분하기" data-toc-modified-id="(3)-train-set과-test-set으로-구분하기-1.0.1.3"><span class="toc-item-num">1.0.1.3&nbsp;&nbsp;</span>(3) train set과 test set으로 구분하기</a></span></li><li><span><a href="#(4)-네거티브샘플링-적용하기" data-toc-modified-id="(4)-네거티브샘플링-적용하기-1.0.1.4"><span class="toc-item-num">1.0.1.4&nbsp;&nbsp;</span>(4) 네거티브샘플링 적용하기</a></span></li></ul></li></ul></li></ul></li></ul></div>

In [ ]:
# for col, z in inputs.items():
#     print(col, z)
#     embedding_mlr_list[col] = Embedding(input_dim=data[col].nunique(), output_dim=1, name=f"Embedding_{col}")(z)

# embedding_mlr_list = []
# for col in data.columns:
#     if col != "cnt":
#         embedding_mlr_list.append(Embedding(input_dim= data[col].nunique(), output_dim=1, name=f"{col}_lr_emb")(inputs[col]))

# class LR(Layer):
#     def build(self, input_shape):
#         # 레이어 연결시 0으로 초기화된 학습 가중치 생성
#         self.b = self.add_weight(shape=(1, ), initializer="zeros", trainable=True)
#         super().build(input_shape)
    
#     def call(self, inputs, **kwargs):
#         logits = tf.add_n(inputs) + self.b
#         return logits

# # Order-2 Part
# dim = 8
# embedding_fm_list = []
# for col in data.columns:
#     if col != "cnt":
# #         fm_embedding_mlr_list.append(Embedding(input_dim= data[col].nunique(), output_dim=dim, name=f"{col}_fm_emb")(inputs[col]))
#         print(Embedding(input_dim= data[col].nunique(), output_dim=dim, name=f"{col}_fm_emb").get_weights)

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.layers import Layer, Dense, Flatten, Dropout, Concatenate, Add, Dot, Multiply, Reshape, Activation, BatchNormalization, SimpleRNNCell, RNN, SimpleRNN, LSTM, Embedding, Bidirectional, TimeDistributed, Conv1D, Conv2D, MaxPool1D, MaxPool2D, GlobalMaxPool1D, GlobalMaxPool2D, AveragePooling1D, AveragePooling2D, GlobalAveragePooling1D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError, BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy, CosineSimilarity
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import linear, sigmoid, relu
from tensorflow.keras.initializers import RandomNormal, glorot_uniform, he_uniform, Constant
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sb
import sklearn

plt.style.use("dark_background")

In [6]:
data = pd.read_csv("./Datasets/Frappe/frappe.csv")
metadata = pd.read_csv("./Datasets/Frappe/frappe_meta.csv", sep="\t")

In [7]:
# "특정 상황에서 고객이 한번 쯤 사용해볼 것 같은 어플을 찾는 추천시스템"으로 목표를 설정해 보겠습니다. 즉, 특정 상황에서 유저가 한번도 사용하지 않았던 어플 중 가장 관심있어 할만한 어플을 찾아주는 문제로 정의할 수 있습니다.
# 1번 이상 로그가 있는 Context에 대해서는 모두 "클릭을 했다" 라는 의미로 `True`로 변경해줍니다.
data["cnt"] = True

# Modeling

In [35]:
inputs = dict()
embedding_mlr_list = list()
embedding_fm_list = list()
for col in data.columns:
    if col != "cnt":
        input_dim = data[col].nunique()
        z = Input(shape=(), name=f"Input_{col}")
        inputs[col] = z

        # Order-1 Part
        z = Embedding(input_dim=input_dim, output_dim=1, name=f"Embedding_mlr_{col}")(z)
        embedding_mlr_list.append(z)
        
        # Order-2 Part
        output_dim = 8
        z = Embedding(input_dim=input_dim, output_dim=output_dim, name=f"Embedding_fm_{col}")(z)
        embedding_fm_list.append(z)

logits_mlr = tf.Variable((0.,)) + tf.math.add_n(embedding_mlr_list)

# Order-2 Part
z = tf.stack(embedding_fm_list, axis=1)
logits_fm = 1/2*tf.math.reduce_sum(tf.math.square(tf.math.reduce_sum(z, axis=1)) - tf.math.reduce_sum(tf.math.square(z), axis=1), axis=1, keepdims=True)

### 3. 데이터 구성하기
#### (1) 명목형 데이터의 인덱싱 표 만들기
- 명목형 데이터에서 각 값을 임베딩 행렬에서 몇 번째 벡터를 가르키는지를 지칭하는 임베딩 인덱스로 바꾸어 주기 위해서, 인덱싱 표를 생성합니다.

In [145]:
cat2idx = {}
cat2size = {}
for col in data.columns:
    if col != "cnt":
        data[col] = data[col].astype("category")
        
        cat2idx[col] = {cat:idx for idx, cat in enumerate(data[col].cat.categories)}
        cat2size[col] = len(data[col].cat.categories)

In [146]:
print(cat2idx)

{'user': {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76, 78: 77, 79: 78, 80: 79, 81: 80, 82: 81, 83: 82, 84: 83, 85: 84, 86: 85, 87: 86, 88: 87, 89: 88, 90: 89, 91: 90, 92: 91, 93: 92, 94: 93, 95: 94, 96: 95, 97: 96, 98: 97, 99: 98, 100: 99, 101: 100, 102: 101, 103: 102, 104: 103, 105: 104, 106: 105, 107: 106, 108: 107, 109: 108, 110: 109, 111: 110, 112: 111, 113: 112, 114: 113, 115: 114, 116: 115, 117: 116, 118: 117, 119: 118, 120: 119, 121: 120,

In [147]:
cat2size

{'user': 792,
 'item': 2072,
 'daytime': 7,
 'weekday': 7,
 'isweekend': 2,
 'homework': 3,
 'cost': 2,
 'weather': 9,
 'country': 78,
 'city': 223}

#### (2) 각 column의 값을 embedding matrix의 index로 변경하기

In [148]:
for col, dic in cat2idx.items():
    data[col] = data[col].map(dic)

#### (3) train set과 test set으로 구분하기

In [171]:
train, test = sklearn.model_selection.train_test_split(data, train_size=0.8)

#### (4) 네거티브샘플링 적용하기
현재 해당 데이터는 특정 컨택스트에서 고객의 앱에 대한 사용 여부로서 "Positive" 데이터만 존재하고 있습니다. 즉, 고객 행동의 긍정적인 행동만 기록되어있다는 것입니다. 우리는 고객이 클릭하지 않은 앱에 대한 사용 여부, 즉 Positive 데이터 한개당 Negative 데이터 3개씩을 생성하되 item 이외의 다른 feature는 고정하고 item을 무작위로 결정하는 식으로 진행해보도록 하겠습니다.

In [184]:
def do_neg_sampling(df, ratio=3):
    neg_sampling = pd.concat([df]*ratio)
    neg_sampling["item"] = df["item"].sample(n=len(neg_sampling), replace=True).values
    # "네거티브 데이터"는 카운트가 존재하지 않음
    neg_sampling["cnt"] = 0

    sampling = pd.concat([df, neg_sampling])
    sampling = sampling.drop_duplicates(sampling.columns.drop("cnt"))
    return sampling

In [185]:
train_sampling = do_neg_sampling(train)
test_sampling = do_neg_sampling(test)

- (3) Input 과 Target 분리
- 데이터 중에서 종속 변수가 되는 요소들( X )과 독립 변수가 되는 클릭 유무을 나누도록 하겠습니다.

In [187]:
train_x = train.drop(["cnt"], axis=1)
train_y = train[["cnt"]]

test_x = test.drop(["cnt"], axis=1)
test_y = test[["cnt"]]

- (4) 텐서로 바꾸어 주기

In [207]:
batch_size = 512

train_set = tf.data.Dataset.from_tensor_slices(({k:v.values.astype("int16") for k, v in train_x.iteritems()}, train_y.values)).shuffle(len(train_x)).batch(batch_size, drop_remainder=True)
test_set = tf.data.Dataset.from_tensor_slices(({k:v.values.astype("int16") for k, v in test_x.iteritems()}, test_y.values)).shuffle(len(test_x)).batch(batch_size, drop_remainder=True)

In [208]:
train_set

<BatchDataset shapes: ({user: (512,), item: (512,), daytime: (512,), weekday: (512,), isweekend: (512,), homework: (512,), cost: (512,), weather: (512,), country: (512,), city: (512,)}, (512, 1)), types: ({user: tf.int16, item: tf.int16, daytime: tf.int16, weekday: tf.int16, isweekend: tf.int16, homework: tf.int16, cost: tf.int16, weather: tf.int16, country: tf.int16, city: tf.int16}, tf.int64)>

In [211]:
train_x

,user,item,daytime,weekday,isweekend,homework,cost,weather,country,city
38883,119,56,3,3,0,1,0,8,75,0
35512,104,19,1,4,1,1,0,0,75,181
80882,355,506,0,3,0,1,0,7,55,148
20930,64,190,1,0,0,0,0,0,17,0
93222,770,283,2,5,1,1,0,7,75,0
...,...,...,...,...,...,...,...,...,...,...
31665,90,23,3,2,0,1,0,3,12,0
89659,501,462,0,2,0,1,0,0,75,173
15745,48,36,2,1,1,1,1,7,67,0
43259,133,100,4,1,1,2,0,0,75,0
